# IMDB Sentiment Analysis

The data is split evenly with 25k reviews intended for training and 25k for testing your classifier. Moreover, each set has 12.5k positive and 12.5k negative reviews.

IMDb lets users rate movies on a scale from 1 to 10. To label these reviews the curator of the data labeled anything with ≤ 4 stars as negative and anything with ≥ 7 stars as positive. Reviews with 5 or 6 stars were left out.

**Import the required libraries**

In [6]:
import numpy as np
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings("ignore")

**Load Data**

In [7]:
reviews_train = []
for line in open('./full_train.txt', 'r', encoding='latin1'):    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('./full_test.txt', 'r', encoding='latin1'):
    reviews_test.append(line.strip())

**See one of the elements in the list**

In [8]:
##### CODE #####
reviews_train[5]

"This isn't the comedic Robin Williams, nor is it the quirky/insane Robin Williams of recent thriller fame. This is a hybrid of the classic drama without over-dramatization, mixed with Robin's new love of the thriller. But this isn't a thriller, per se. This is more a mystery/suspense vehicle through which Williams attempts to locate a sick boy and his keeper.<br /><br />Also starring Sandra Oh and Rory Culkin, this Suspense Drama plays pretty much like a news report, until William's character gets close to achieving his goal.<br /><br />I must say that I was highly entertained, though this movie fails to teach, guide, inspect, or amuse. It felt more like I was watching a guy (Williams), as he was actually performing the actions, from a third person perspective. In other words, it felt real, and I was able to subscribe to the premise of the story.<br /><br />All in all, it's worth a watch, though it's definitely not Friday/Saturday night fare.<br /><br />It rates a 7.7/10 from...<br />

The raw text is pretty messy for these reviews so before we can do any analytics we need to clean things up


**Use Regular expressions to remove the non text characters, and the html tags**

In [9]:
import re #regular expressions

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews] #.sub() sustituir
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [10]:
reviews_train_clean[5]

"this isn't the comedic robin williams nor is it the quirky insane robin williams of recent thriller fame this is a hybrid of the classic drama without over dramatization mixed with robin's new love of the thriller but this isn't a thriller per se this is more a mystery suspense vehicle through which williams attempts to locate a sick boy and his keeper also starring sandra oh and rory culkin this suspense drama plays pretty much like a news report until william's character gets close to achieving his goal i must say that i was highly entertained though this movie fails to teach guide inspect or amuse it felt more like i was watching a guy williams as he was actually performing the actions from a third person perspective in other words it felt real and i was able to subscribe to the premise of the story all in all it's worth a watch though it's definitely not friday saturday night fare it rates a   from the fiend "

In [11]:
len(reviews_train_clean)

25000

# Vectorization
In order for this data to make sense to our machine learning algorithm we’ll need to convert each review to a numeric representation, which we call vectorization.

The simplest form of this is to create one very large matrix with one column for every unique word in your corpus (where the corpus is all 50k reviews in our case). Then we transform each review into one row containing 0s and 1s, where 1 means that the word in the corpus corresponding to that column appears in that review. That being said, each row of the matrix will be very sparse (mostly zeros). This process is also known as one hot encoding. Use the *CountVectorizer* method.

In [12]:
# Mapeamos palabra a numeros para posicionarlas en un espacio multidimensional

In [13]:
##### CODE #####
from sklearn.feature_extraction.text import CountVectorizer
#Si aparece una palabra en una reseña. le pone un 1.CountVectorizer
#Da igual que aparezca 100 veces, no cuenta

baseline_vectorizer = CountVectorizer(binary= True)
baseline_vectorizer.fit(reviews_train_clean)

#reseñas en formato vector de palabras
X_baseline = baseline_vectorizer.transform(reviews_train_clean)
X_test_baseline = baseline_vectorizer.transform(reviews_test_clean)

In [14]:
##### CODE #####
print(X_baseline.shape) #tupla (número de reseñas, número de palabras distintas después de limpiar)
baseline_vectorizer.vocabulary_

(25000, 87063)


{'bromwell': 9819,
 'high': 35211,
 'is': 39472,
 'cartoon': 11686,
 'comedy': 14754,
 'it': 39642,
 'ran': 61772,
 'at': 4537,
 'the': 76725,
 'same': 66138,
 'time': 77626,
 'as': 4211,
 'some': 71188,
 'other': 54861,
 'programs': 60156,
 'about': 284,
 'school': 67025,
 'life': 44297,
 'such': 74177,
 'teachers': 75997,
 'my': 51490,
 'years': 86260,
 'in': 37733,
 'teaching': 76000,
 'profession': 60088,
 'lead': 43605,
 'me': 47976,
 'to': 77922,
 'believe': 6894,
 'that': 76671,
 'satire': 66462,
 'much': 51018,
 'closer': 14125,
 'reality': 62242,
 'than': 76643,
 'scramble': 67253,
 'survive': 74812,
 'financially': 27904,
 'insightful': 38615,
 'students': 73768,
 'who': 84627,
 'can': 11132,
 'see': 67695,
 'right': 64476,
 'through': 77369,
 'their': 76762,
 'pathetic': 56362,
 'pomp': 58787,
 'pettiness': 57341,
 'of': 53843,
 'whole': 84639,
 'situation': 69959,
 'all': 2038,
 'remind': 63323,
 'schools': 67049,
 'knew': 42226,
 'and': 2762,
 'when': 84450,
 'saw': 66588,

In [15]:
##### CODE #####
vectorizer_c = CountVectorizer()
vectorizer_c.fit(reviews_train_clean)

# reseñas en formato vector de palabras
X_baseline_c = vectorizer_c.transform(reviews_train_clean)

In [16]:
print(X_baseline_c.shape)
print(len(vectorizer_c.get_feature_names()))

(25000, 87063)
87063


# Train a Baseline Model

Train a Logistic Regression model after transforming the data with CountVectorized

* They’re easy to interpret
* Linear models tend to perform well on sparse datasets like this one
* They learn very fast compared to other algorithms.

Test models with C values of [0.01, 0.05, 0.25, 0.5, 1] and see wich is the best value for C, and calculate the accuracy

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#Los comentarios vienen ordenados. los primeros 12500 son negativos
# A test le ocurre lo mismo
target = [1 if i < 12500 else 0 for i in range(25000)]


def train_model(X_TRAIN, X_TEST):
    
    X_train, X_val, y_train, y_val = train_test_split(X_TRAIN,
                                                      target,
                                                      train_size = 0.75,
                                                      random_state=42)

    for c in [0.01, 0.05, 0.25, 0.5, 1]:

        lr = LogisticRegression(C=c)
        lr.fit(X_train, y_train)
        print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_val, lr.predict(X_val))))

    final_model = LogisticRegression(C=0.05)
    final_model.fit(X_TRAIN, target)

    print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_TEST)))
    
train_model(X_baseline, X_test_baseline)

Accuracy for C=0.01: 0.86864
Accuracy for C=0.05: 0.87584
Accuracy for C=0.25: 0.87808
Accuracy for C=0.5: 0.8752
Accuracy for C=1: 0.87376
Final Accuracy: 0.88188


# Remove Stop Words

Stop words are the very common words like ‘if’, ‘but’, ‘we’, ‘he’, ‘she’, and ‘they’. We can usually remove these words without changing the semantics of a text and doing so often (but not always) improves the performance of a model. Removing these stop words becomes a lot more useful when we start using longer word sequences as model features (see n-grams below).

Before we apply the CountVectorized, lets remove the stopwords, included in nltk.corpus (nltk ==> natural language tool kit)

Then apply the CountVectorizer, and train the Logistic regression model and obtain the accuracy.

In [22]:
##### CODE #####
# Para visualizar las stopwords de inglés
from nltk.corpus import stopwords
stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [23]:
##### CODE #####
#para visualizar en español
stopwords.words('spanish')[:10]

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se']

In [24]:
##### CODE #####

In [25]:
len([word for word in reviews_train_clean[5].split()])

168

In [26]:
len([word for word in reviews_train_clean[5].split() if word not in stopwords.words('english')])

91

In [27]:
from nltk.corpus import stopwords

#Aplicamos la eliminación de las palabras directamente sobre las reseñas
english_stop_words = stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        
        # Para cada review elimina las stopwords, y separa todas las palabras por espacio
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_test = remove_stop_words(reviews_test_clean)

In [28]:
##### CODE #####
cv = CountVectorizer(binary= True)
cv.fit(no_stop_words_train)
X = cv.transform(no_stop_words_train)
X_test = cv.transform(no_stop_words_test)

train_model(X, X_test)

Accuracy for C=0.01: 0.86656
Accuracy for C=0.05: 0.87632
Accuracy for C=0.25: 0.8792
Accuracy for C=0.5: 0.87776
Accuracy for C=1: 0.87632
Final Accuracy: 0.87936


In [29]:
print(X_baseline.shape)
print(X.shape)
print("Stop words eliminadas:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 87046)
Stop words eliminadas: 17


In [30]:
##### CODE #####
#También podemos eliminar las stopwords directamente por parámetro
cv = CountVectorizer(binary= True,
                     stop_words= english_stop_words) #Existe en la clase un parámetro 'stop_words' para eliminar las palabras que indiques en la lista 

cv.fit(reviews_train_clean)

X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

train_model(X, X_test)

Accuracy for C=0.01: 0.8664
Accuracy for C=0.05: 0.876
Accuracy for C=0.25: 0.8808
Accuracy for C=0.5: 0.87824
Accuracy for C=1: 0.87696
Final Accuracy: 0.87976


In [31]:
print(X_baseline.shape)
print(X.shape)
print("Stop words eliminadas:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 86918)
Stop words eliminadas: 145


**Note:** In practice, an easier way to remove stop words is to just use the stop_words argument with any of scikit-learn’s ‘Vectorizer’ classes. If you want to use NLTK’s full list of stop words you can do stop_words='english’. In practice I’ve found that using NLTK’s list actually decreases my performance because its too expansive, so I usually supply my own list of words. For example, stop_words=['in','of','at','a','the'] .

A common next step in text preprocessing is to normalize the words in your corpus by trying to convert all of the different forms of a given word into one. Two methods that exist for this are Stemming and Lemmatization.

# Stemming

Stemming is considered to be the more crude/brute-force approach to normalization (although this doesn’t necessarily mean that it will perform worse). There’s several algorithms, but in general they all use basic rules to chop off the ends of words.

NLTK has several stemming algorithm implementations. We’ll use the Porter stemmer. Most used:
* PorterStemmer
* SnowballStemmer

Apply a PoterStemmer, vectorize, and train the model again

In [32]:
##### CODE #####

"""
El stemmer se aplica sobre cada palabra. Las recortas eliminando plurales y tiempos verbales.
"""
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

plurals = ['caresses', 'flies', 'dies', 'mules', 'denied',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted'] #

singles = [stemmer.stem(plural) for plural in plurals]
print(" ".join(singles))

##### CODE #####

caress fli die mule deni die agre own humbl size meet state siez item sensat tradit refer colon plot


In [33]:
##### CODE #####
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

plurals = ['caresses', 'flies', 'dies', 'mules', 'denied',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']
singles = [stemmer.stem(plural) for plural in plurals]

print(' '.join(singles))

caress fli die mule deni die agre own humbl size meet state siez item sensat tradit refer colon plot


In [34]:
from nltk.stem.snowball import SnowballStemmer
##### CODE #####
stemmer = SnowballStemmer('spanish')

plurals = ['corriendo', 'casas', 'playa', 'volando', 'volar', 'volveré']
singles = [stemmer.stem(plural) for plural in plurals]

print(' '.join(singles))

corr cas play vol vol volv


In [35]:
##### CODE #####
#los stemmer no eliminan palabras sólo quitán sufijos
#Y se consigue más número de palabra iguales (coincide la "raiz")
def get_stemmed_text(corpus):
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]




stemmed_reviews_train = get_stemmed_text(reviews_train_clean)
stemmed_reviews_test = get_stemmed_text(reviews_test_clean)

cv = CountVectorizer(binary=True)
cv.fit(stemmed_reviews_train)
X_stem = cv.transform(stemmed_reviews_train)
X_test = cv.transform(stemmed_reviews_test)

train_model(X_stem, X_test)

Accuracy for C=0.01: 0.8704
Accuracy for C=0.05: 0.87776
Accuracy for C=0.25: 0.87744
Accuracy for C=0.5: 0.87344
Accuracy for C=1: 0.86992
Final Accuracy: 0.87736


In [36]:
print(X_baseline.shape)
print(X_stem.shape)
print("Diff X normal y X tras stemmer y vectorización:", X_baseline.shape[1] - X_stem.shape[1])

(25000, 87063)
(25000, 66852)
Diff X normal y X tras stemmer y vectorización: 20211


# Lemmatization

Lemmatization works by identifying the part-of-speech of a given word and then applying more complex rules to transform the word into its true root.

In [37]:
##### CODE #####
import nltk as nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pilar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
##### CODE #####

plurals = ['caresses', 'flies', 'dies', 'mules', 'studies',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']
singles = [lemmatizer.lemmatize(plural) for plural in plurals]

print(' '.join(singles))

caress fly dy mule study died agreed owned humbled sized meeting stating siezing itemization sensational traditional reference colonizer plotted


In [39]:
def get_lemmatized_text(corpus):
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

# Lematizamos las reviews
lemmatized_reviews_train = get_lemmatized_text(reviews_train_clean)
lemmatized_reviews_test = get_lemmatized_text(reviews_test_clean)

# Vectorizamos con conteo tras lematizar
cv = CountVectorizer(binary=True)
cv.fit(lemmatized_reviews_train)
X = cv.transform(lemmatized_reviews_train)
X_test = cv.transform(lemmatized_reviews_test)

train_model(X, X_test)

Accuracy for C=0.01: 0.86528
Accuracy for C=0.05: 0.8752
Accuracy for C=0.25: 0.87392
Accuracy for C=0.5: 0.87344
Accuracy for C=1: 0.87152
Final Accuracy: 0.87956


In [40]:
print(X_baseline.shape)
print(X.shape)
print("Diff X normal y X tras lematizador y vectorización:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 80326)
Diff X normal y X tras lematizador y vectorización: 6737


# n-grams

We can potentially add more predictive power to our model by adding two or three word sequences (bigrams or trigrams) as well. For example, if a review had the three word sequence “didn’t love movie” we would only consider these words individually with a unigram-only model and probably not capture that this is actually a negative sentiment because the word ‘love’ by itself is going to be highly correlated with a positive review.

The scikit-learn library makes this really easy to play around with. Just use the ngram_range argument with any of the ‘Vectorizer’ classes.

In [41]:
##### CODE #####
from nltk import ngrams

sentence = "this is foo bar"

two = ngrams(sentence.split(),2)
three = ngrams(sentence.split(),3)

for grams in two:
  print(grams)
print('###############')
for grams in three:
  print(grams)

##### CODE #####
"""
Puede ser bigrams si ngram_range = (2,2), o trigramas...
Algunas palabras las elimina, como 'a'. Cuidado con eso a la hora de hacer el 
conteo ngram_range =(1,3) significa palabras por separado, los bigramas y los trigramas

Ojo que esto aumenta muchísimo el espacio de features
"""
ngrams_vectorizer = CountVectorizer(binary= True,
                                      ngram_range=(1,3))

vector = ngrams_vectorizer.fit_transform([sentence]).toarray()
print()
print(vector)
print(len(vector[0]))

('this', 'is')
('is', 'foo')
('foo', 'bar')
###############
('this', 'is', 'foo')
('is', 'foo', 'bar')

[[1 1 1 1 1 1 1 1 1]]
9


In [42]:
##### CODE #####
ngram_vectorizer = CountVectorizer(binary=True,
                                    ngram_range=(1,2))

ngram_vectorizer.fit(reviews_train_clean)

X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

train_model(X, X_test)

Accuracy for C=0.01: 0.87696
Accuracy for C=0.05: 0.88384
Accuracy for C=0.25: 0.8872
Accuracy for C=0.5: 0.88768


KeyboardInterrupt: 

In [ ]:
print(X_baseline.shape)
print(X.shape)
print("Diff X normal y X tras lematizador y vectorización:", X_baseline.shape[1] - X.shape[1])

# TF-IDF

Another common way to represent each document in a corpus is to use the tf-idf statistic (term frequency-inverse document frequency) for each word, which is a weighting factor that we can use in place of binary or word count representations.

There are several ways to do tf-idf transformation but in a nutshell, **tf-idf aims to represent the number of times a given word appears in a document (a movie review in our case) relative to the number of documents in the corpus that the word appears in** — where words that appear in many documents have a value closer to zero and words that appear in less documents have values closer to 1.

**Note:** Now that we’ve gone over n-grams, when I refer to ‘words’ I really mean any n-gram (sequence of words) if the model is using an n greater than one.

In [ ]:
##### CODE #####
"""
ln(N + 1 / df +1) +1
Cuanto más común, menor es el TFIDF. Cuanto más rara, mayor
"""

#Número de documentos
N = 3

#Numero de veces que aparece la palabra
df = 2

1 + np.log((N + 1)/(df + 1))

In [ ]:
##### CODE #####
from sklearn.feature_extraction.text import TfidfVectorizer

"""
Cuanto más común, más bajo es el TfidfVectorizer
"""
sent1 = "My name is Ralph"
sent2 = "Ralph is happy"
sent3 = "Ralph"

test = TfidfVectorizer()
test.fit_transform([sent1,sent2, sent3])
print(test.idf_)
print(test.get_feature_names())

"""

tfidf_vectorizer.fit(reviews_train_clean)
X = tfidf_vectorizer.transform(reviews_train_clean)
X_test = tfidf_vectorizer.transform(reviews_test_clean)


train_model(X, X_test)
"""

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

tfidf_vectorizer = TfidfVectorizer()

tfidf_vectorizer.fit(reviews_train_clean)
X = tfidf_vectorizer.transform(reviews_train_clean)
X_test = tfidf_vectorizer.transform(reviews_test_clean)

train_model(X, X_test)

# Support Vector Machines (SVM)

Recall that linear classifiers tend to work well on very sparse datasets (like the one we have). Another algorithm that can produce great results with a quick training time are Support Vector Machines with a linear kernel.

Build a model with an n-gram range from 1 to 2:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# SVM con bigramas
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))
    
final_svm_ngram = LinearSVC(C=0.01)
final_svm_ngram.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_svm_ngram.predict(X_test)))


# Final Model

Removing a small set of stop words along with an n-gram range from 1 to 3 and a linear support vector classifier shows the best results.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC


##### CODE #####
stop_words = ['in', 'of', 'at', 'a', 'the']
ngram_vectorizer = CountVectorizer(binary= True,
                                   ngram_range=(1,3),
                                   stop_words= stop_words)

ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

for c in [0.001, 0.005, 0.01, 0.05, 0.1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))

    
final = LinearSVC(C=0.01)
final.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final.predict(X_test)))


# Top Postitive and Negative Features

Obtain the most important features of the model.

In [18]:
##### CODE ####
cv = CountVectorizer(binary= True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)

log_reg = LogisticRegression(C = 0.5)
log_reg.fit(X, target)

final_model = log_reg

#importancia de los coeficientes. En total, todas las palabras vectorizadas
print(len(final_model.coef_[0]))

#Cada coeficiente va asociado a una palabra
cv.get_feature_names()

# Montamos un diccionario con palabra -> coeficiente
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}

87063


In [19]:
feature_to_coef

027358317622930116,
 'accentcourtesy': 0.03274539513581589,
 'accented': 0.021386262450647085,
 'accenthe': -0.0032129614449121205,
 'accentit': -0.03605147663282439,
 'accentlike': 0.03274539513581589,
 'accents': -0.24490449936745923,
 'accentsomeone': -0.0008323722420750297,
 'accentuate': 0.06330820450952748,
 'accentuated': 0.06515384935098743,
 'accentuates': 0.08460252771205184,
 'accentuating': -0.01022260062070583,
 'accentuation': 0.0015654923513505404,
 'accept': 0.08709325813854711,
 'acceptable': -0.15006225756259578,
 'acceptablethe': 0.07278614712921161,
 'acceptably': 0.16945751143835486,
 'acceptance': 0.0834963416907583,
 'acceptation': 0.0011710872086697405,
 'accepted': -0.3427061036632185,
 'acceptence': 0.0007298621491786402,
 'accepting': 0.1882251123071673,
 'acception': 0.028396929640109817,
 'accepts': 0.13696767838444596,
 'accesible': -0.0002158234966006485,
 'access': 0.12457019526419127,
 'accessability': 0.004505378021177838,
 'accessed': -0.0132667456350

In [20]:
##### CODE #####
for best_positive in sorted(
                            feature_to_coef.items(), 
                            key= lambda x: x[1], 
                            reverse= True)[:5]:
    print(best_positive)

print('#############################')

for best_negative in sorted(feature_to_coef.items(), key= lambda x: x[1])[:5]:
    print(best_negative)

('excellent', 1.3796382341255753)
('refreshing', 1.2804007428476882)
('perfect', 1.201765417928455)
('superb', 1.1359048015765132)
('appreciated', 1.1338062594644855)
#############################
('worst', -2.060193118611848)
('waste', -1.9166831490720768)
('disappointment', -1.675687288820553)
('poorly', -1.6570778362602712)
('awful', -1.535364979859899)


In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

print(model)

y_pred = model.predict(X_val)

accuracy = accuracy_score(y_val, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
